<a href="https://colab.research.google.com/github/SEOUL-ABSS/Sonya/blob/main/ai%ED%83%90%EC%A7%802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 환경 설정 및 필수 라이브브러리 임포트


프로젝트에 필요한 모든 라이브러리를 가져오고, 시각화(Matplotlib)를 위한 한글 폰트 설정 및 전역 변수를 정의합니다.

In [1]:
# ==============================================================================
# 1. 환경 설정 및 필수 라이브러리 임포트
# ==============================================================================
print("1. 환경 설정 및 라이브러리 임포트 중...")

# --- 필수 라이브러리 임포트 ---
!pip install boto3
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from pathlib import Path

print("라이브러리 임포트 완료.")

# --- Matplotlib 한글 폰트 설정 ---
print("\nMatplotlib 한글 폰트 설정 중...")
try:
    !sudo apt-get -qq update
    !sudo apt-get -qq install -y fonts-nanum
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if os.path.exists(font_path):
        fm.fontManager.addfont(font_path)
        plt.rc('font', family='NanumGothic')
        plt.rcParams['axes.unicode_minus'] = False
        print("Matplotlib 폰트 설정 완료: NanumGothic")
    else:
        print(f"경고: 폰트 파일 '{font_path}'를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
except Exception as e:
    print(f"Matplotlib 폰트 설정 중 오류 발생: {e}. 기본 폰트를 사용합니다.")

# --- 전역 상수 정의 ---
print("\n전역 상수 정의 중...")
YAMNET_SAMPLE_RATE = 16000
YAMNET_EMBEDDING_DIM = 1024
VGGISH_EMBEDDING_DIM = 128
PANNS_EMBEDDING_DIM = YAMNET_EMBEDDING_DIM  # PANNs는 YAMNet을 플레이스홀더로 사용

# 데이터셋 경로
DEEPSHIP_BASE_PATH = '/content/DeepShip'
MBARI_NOISE_BASE_DIR = '/content/MBARI_noise_data'

# 처리할 모델 목록
MODELS_TO_PROCESS = ['YAMNet', 'PANNs', 'VGGish']

print("전역 상수 정의 완료.")

1. 환경 설정 및 라이브러리 임포트 중...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.1 MB/s eta 0:00:00
라이브러리 임포트 완료.

Matplotlib 한글 폰트 설정 중...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Readi

2. 데이터셋 준비 (다운로드 및 클론)

'ship' 데이터인 DeepShip은 GitHub에서 클론하고, 'noise' 데이터인 MBARI 데이터는 AWS S3에서 다운로드하여 지정된 경로에 준비합니다.

In [2]:
# ==============================================================================
# 2. 데이터셋 준비 (다운로드 및 클론)
# ==============================================================================
def setup_datasets():
    """DeepShip 데이터셋을 클론하고 MBARI 노이즈 데이터를 다운로드합니다."""
    print("\n2. 데이터셋 준비 중...")

    # --- DeepShip 데이터셋 클론 ---
    if not os.path.exists(DEEPSHIP_BASE_PATH):
        print(f"DeepShip 데이터셋 클론 중: {DEEPSHIP_BASE_PATH}")
        !git clone -q https://github.com/irfankamboh/DeepShip.git {DEEPSHIP_BASE_PATH}
        print("DeepShip 데이터셋 클론 완료.")
    else:
        print(f"DeepShip 데이터셋이 이미 존재합니다: {DEEPSHIP_BASE_PATH}")

    # --- MBARI 노이즈 데이터 다운로드 ---
    os.makedirs(MBARI_NOISE_BASE_DIR, exist_ok=True)
    noise_filename = 'MARS-20180101T000000Z-16kHz.wav'
    target_noise_path = os.path.join(MBARI_NOISE_BASE_DIR, noise_filename)

    if not Path(target_noise_path).exists():
        print(f"MBARI 노이즈 데이터 다운로드 중: s3://pacific-sound-16khz/2018/01/{noise_filename}")
        try:
            s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
            bucket = s3.Bucket('pacific-sound-16khz')
            key = f'2018/01/{noise_filename}'
            bucket.download_file(key, target_noise_path)
            print(f"MBARI 노이즈 데이터 다운로드 완료: {target_noise_path}")
        except Exception as e:
            print(f"오류: MBARI 데이터 다운로드 실패: {e}")
    else:
        print(f"MBARI 노이즈 데이터가 이미 존재합니다: {target_noise_path}")

# 데이터셋 준비 함수 실행
setup_datasets()


2. 데이터셋 준비 중...
DeepShip 데이터셋 클론 중: /content/DeepShip
DeepShip 데이터셋 클론 완료.
MBARI 노이즈 데이터 다운로드 중: s3://pacific-sound-16khz/2018/01/MARS-20180101T000000Z-16kHz.wav
MBARI 노이즈 데이터 다운로드 완료: /content/MBARI_noise_data/MARS-20180101T000000Z-16kHz.wav


3. 핵심 기능 함수 정의
데이터 로드, 오디오 전처리, 임베딩 추출, 모델 구축, 학습, 평가 등 반복적으로 사용되는 로직들을 기능별 함수로 명확하게 정의합니다.

In [5]:
# ==============================================================================
# 3. 핵심 기능 함수 정의
# ==============================================================================
print("\n3. 핵심 기능 함수 정의 중...")

def load_and_prepare_dataset(deepship_path, noise_data_dir):
    """DeepShip과 노이즈 데이터를 로드하고 훈련/테스트 세트로 분할합니다."""
    all_audio_paths, all_labels = [], []

    # --- DeepShip('ship') 데이터 로드 ---
    deepship_acoustic_path = os.path.join(deepship_path, 'Acoustic_data')
    if os.path.exists(deepship_acoustic_path):
        print(f"DeepShip 데이터셋에서 'ship' 오디오 파일 수집 중: {deepship_acoustic_path}")

        # Debugging: Print directory structure (keep for now to confirm fix)
        print("\nDeepShip Acoustic_data 디렉토리 구조 확인:")
        try:
            for root, dirs, files in os.walk(deepship_acoustic_path):
                level = root.replace(deepship_acoustic_path, '').count(os.sep)
                indent = ' ' * 4 * (level)
                print(f'{indent}{os.path.basename(root)}/')
                subindent = ' ' * 4 * (level + 1)
                for f in files:
                    if f.endswith('.wav'):
                        print(f'{subindent}{f}')
        except Exception as e:
            print(f"디렉토리 구조 확인 중 오류 발생: {e}")
        print("-------------------------------------\n")


        # Modified logic: Collect .wav files within 'Train' and 'Test' subdirectories
        for root, _, files in os.walk(deepship_acoustic_path):
            for file in files:
                if file.endswith('.wav'):
                    full_path = os.path.join(root, file)
                    # Check if the file is within a 'Train' or 'Test' subdirectory of Acoustic_data
                    relative_path = os.path.relpath(full_path, deepship_acoustic_path)
                    path_parts_relative = relative_path.split(os.sep)

                    # Simplified logic: Check if the path contains 'Train' or 'Test' and is not directly in Acoustic_data
                    if ('Train' in path_parts_relative or 'Test' in path_parts_relative) and len(path_parts_relative) > 1:
                         all_audio_paths.append(full_path)
                         all_labels.append('ship')
                         # print(f"Found ship audio: {full_path}") # Uncomment for debugging

    else:
        print(f"경고: DeepShip Acoustic_data 경로를 찾을 수 없습니다: {deepship_acoustic_path}")


    # --- MBARI('noise') 데이터 로드 ---
    if os.path.exists(noise_data_dir):
        print(f"노이즈 데이터 수집 중: {noise_data_dir}")
        for file in os.listdir(noise_data_dir):
            if file.endswith('.wav'):
                full_path = os.path.join(noise_data_dir, file)
                all_audio_paths.append(full_path)
                all_labels.append('noise')
                # print(f"Found noise audio: {full_path}") # Uncomment for debugging
    else:
        print(f"경고: 노이즈 데이터 경로를 찾을 수 없습니다: {noise_data_dir}")

    if not all_audio_paths or len(np.unique(all_labels)) < 2:
        print(f"\n오류: 'ship'과 'noise' 클래스 모두에 대한 데이터가 부족합니다. 총 샘플 수: {len(all_audio_paths)}, 클래스 수: {len(np.unique(all_labels)) if all_labels else 0}. 처리를 중단합니다.")
        return [], [], [], [], None, False, 0

    print(f"\n데이터 수집 완료. 총 샘 샘플 수: {len(all_audio_paths)}")
    print(f"클래스 분포: {pd.Series(all_labels).value_counts().to_dict()}")

    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(all_labels)
    num_classes = len(label_encoder.classes_)

    X_train_paths, X_test_paths, y_train, y_test = train_test_split(
        all_audio_paths, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
    )
    print(f"데이터 분할 완료. 훈련: {len(X_train_paths)}, 테스트: {len(X_test_paths)}")
    return X_train_paths, X_test_paths, y_train, y_test, label_encoder, True, num_classes

def load_audio_models():
    """TensorFlow Hub에서 오디오 모델들을 로드합니다."""
    models = {}
    handles = {
        'YAMNet': 'https://tfhub.dev/google/yamnet/1',
        'PANNs': 'https://tfhub.dev/google/yamnet/1',  # Placeholder
        'VGGish': 'https://tfhub.dev/google/vggish/1'
    }
    for name, handle in handles.items():
        try:
            print(f"  {name} 모델 로드 중...")
            models[name] = hub.load(handle)
        except Exception as e:
            print(f"  오류: {name} 모델 로드 실패: {e}")
            models[name] = None
    return models

def extract_embeddings(paths, labels, model_name, model_object):
    """주어진 오디오 경로 목록에서 임베딩을 추출합니다."""
    embeddings, filtered_labels = [], []

    def _get_embedding(path, model):
        try:
            waveform, sr = sf.read(path, dtype='float32')
            if sr != YAMNET_SAMPLE_RATE:
                waveform = librosa.resample(y=waveform, orig_sr=sr, target_sr=YAMNET_SAMPLE_RATE)
            if waveform.ndim > 1:
                waveform = np.mean(waveform, axis=1)

            if model_name == 'VGGish':
                return np.mean(model(tf.constant(waveform, dtype=tf.float32)), axis=0)
            else: # YAMNet and PANNs (placeholder)
                _, embedding, _ = model(tf.constant(waveform, dtype=tf.float32))
                return np.mean(embedding, axis=0)
        except Exception as e:
            print(f"    오류: {path} 임베딩 추출 실패: {e}")
            return None

    for i, path in enumerate(paths):
        embedding = _get_embedding(path, model_object)
        if embedding is not None:
            embeddings.append(embedding.numpy())
            filtered_labels.append(labels[i])
        if (i + 1) % 100 == 0:
            print(f"    {i+1}/{len(paths)} 파일 처리 완료.")

    return np.array(embeddings), np.array(filtered_labels)

def build_classifier(embedding_dim, num_classes):
    """분류 헤드 모델을 구축합니다."""
    input_layer = Input(shape=(embedding_dim,))
    x = Dense(128, activation='relu')(input_layer)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate_and_visualize(model_name, model, history, X_test, y_test, y_test_filtered, label_encoder):
    """모델 성능을 평가하고 결과를 시각화합니다."""
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\n--- {model_name} 모델 평가 결과 ---")
    print(f"  테스트 손실: {loss:.4f}")
    print(f"  테스트 정확도: {accuracy:.4f}")

    y_pred = np.argmax(model.predict(X_test), axis=1)

    print("\n  분류 리포트:")
    print(classification_report(y_test_filtered, y_pred, target_names=label_encoder.classes_))

    cm = confusion_matrix(y_test_filtered, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(f'{model_name} 혼동 행렬')
    plt.xlabel('예측 레이블')
    plt.ylabel('실제 레이블')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='훈련 정확도')
    plt.plot(history.history['val_accuracy'], label='검증 정확도')
    plt.title(f'{model_name} 훈련 및 검증 정확도')
    plt.xlabel('에폭'); plt.ylabel('정확도'); plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='훈련 손실')
    plt.plot(history.history['val_loss'], label='검증 손실')
    plt.title(f'{model_name} 훈련 및 검증 손실')
    plt.xlabel('에폭'); plt.ylabel('손실'); plt.legend()
    plt.tight_layout(); plt.show()

print("핵심 기능 함수 정의 완료.")


3. 핵심 기능 함수 정의 중...
핵심 기능 함수 정의 완료.


4. 메인 실행 파이프라인


정의된 함수들을 순서대로 호출하여 전체 데이터 처리, 모델 학습 및 평가 파이프라인을 실행합니다. 이 셀 하나만 실행하면 모든 과정이 진행됩니다.

In [4]:
# ==============================================================================
# 4. 메인 실행 파이프라인
# ==============================================================================
print("\n4. 메인 파이프라인 실행 시작...")

# --- 데이터 로드 및 준비 ---
X_train_paths, X_test_paths, y_train_enc, y_test_enc, label_encoder, is_data_prepared, num_classes = \
    load_and_prepare_dataset(DEEPSHIP_BASE_PATH, MBARI_NOISE_BASE_DIR)

if is_data_prepared:
    # --- 오디오 모델 로드 ---
    audio_models = load_audio_models()

    final_results = {}

    # --- 모델별 임베딩 추출, 학습, 평가 ---
    for model_name in MODELS_TO_PROCESS:
        model_object = audio_models.get(model_name)
        if model_object is None:
            print(f"\n--- {model_name} 모델을 로드할 수 없어 건너뜁니다. ---")
            continue

        print(f"\n--- {model_name} 처리 시작 ---")

        # 임베딩 추출
        print(f"  {model_name} 훈련 데이터 임베딩 추출 중...")
        X_train_emb, y_train_filtered = extract_embeddings(X_train_paths, y_train_enc, model_name, model_object)

        print(f"  {model_name} 테스트 데이터 임베딩 추출 중...")
        X_test_emb, y_test_filtered = extract_embeddings(X_test_paths, y_test_enc, model_name, model_object)

        if X_train_emb.size == 0 or X_test_emb.size == 0:
            print(f"  오류: {model_name} 임베딩 추출에 실패하여 학습을 건너뜁니다.")
            continue

        # 레이블 One-Hot 인코딩
        y_train_one_hot = tf.keras.utils.to_categorical(y_train_filtered, num_classes=num_classes)
        y_test_one_hot = tf.keras.utils.to_categorical(y_test_filtered, num_classes=num_classes)

        # 분류기 구축
        embedding_dim = X_train_emb.shape[1]
        classifier = build_classifier(embedding_dim, num_classes)

        # 모델 학습
        history = classifier.fit(
            X_train_emb, y_train_one_hot,
            validation_data=(X_test_emb, y_test_one_hot),
            epochs=50,
            batch_size=32,
            callbacks=[
                EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
            ],
            verbose=1
        )

        # 모델 평가 및 시각화
        evaluate_and_visualize(model_name, classifier, history, X_test_emb, y_test_one_hot, y_test_filtered, label_encoder)

        # 최종 결과 저장
        _, accuracy = classifier.evaluate(X_test_emb, y_test_one_hot, verbose=0)
        final_results[model_name] = accuracy

    # --- 최종 성능 요약 ---
    print("\n--- 최종 모델 성능 비교 ---")
    if final_results:
        for model_name, acc in sorted(final_results.items(), key=lambda item: item[1], reverse=True):
            print(f"  - {model_name}: Test Accuracy = {acc:.4f}")
    else:
        print("  평가된 모델이 없습니다.")

else:
    print("\n데이터 준비에 실패하여 모델 학습 및 평가를 진행하지 않았습니다.")

print("\n모든 파이프라인 실행 완료.")


4. 메인 파이프라인 실행 시작...
경고: DeepShip Acoustic_data 경로를 찾을 수 없습니다: /content/DeepShip/Acoustic_data
노이즈 데이터 수집 중: /content/MBARI_noise_data

오류: 'ship'과 'noise' 클래스 모두에 대한 데이터가 부족합니다. 총 샘플 수: 1, 클래스 수: 1. 처리를 중단합니다.

데이터 준비에 실패하여 모델 학습 및 평가를 진행하지 않았습니다.

모든 파이프라인 실행 완료.
